goal is to get 2022 technics sets inventory listed and calculate super-set and corelation for all set to super-set<br>
to do:
* get id of technic theme from GET /api/v3/lego/themes/ and save a it
* get list of sets from GET /api/v3/lego/sets/ filtered by paramters: technic theme id and year 2022 
* for each set get inventory from GET /api/v3/lego/sets/{set_num}/parts/ and store it in table
* build super-set(max for each part)
* calculate corelation for each of starting sets with super-set

In [37]:
import requests
import json

keys = json.load(open('keys.json'))
API_KEY = '/?key=' + keys['API_KEY']
api_url = 'https://rebrickable.com/api/v3/'

In [41]:
themes_url = api_url + 'lego/themes' + API_KEY
response = requests.get(themes_url).json()
themes = response['results']

for theme in themes:
    if theme['name'] == 'Technic':
        technic_theme_id = str(theme['id'])

technic_theme_id


'1'

In [42]:

theme_id = technic_theme_id
min_year = '2022'
max_year = '2022'
sets_url = api_url + 'lego/sets' + API_KEY + \
    '&theme_id=' + theme_id + '&min_year=' + min_year + '&max_year=' + max_year
response = requests.get(sets_url).json()
sets = response['results']
sets_list = []

for set in sets:
    sets_list.append(set['set_num'])

sets_list


['30433-1',
 '42130-1',
 '42132-1',
 '42133-1',
 '42134-1',
 '42135-1',
 '42136-1',
 '42137-1',
 '42138-1',
 '42139-1',
 '42140-1',
 '42141-1',
 '42141-2',
 '42143-1',
 '42144-1',
 '42145-1',
 '66712-1']

In [45]:
set_numbers, part_numbers, quantities = [], [], []

for set in sets_list:
    inventory_parts_url = api_url + 'lego/sets/' + \
        set + '/parts' + API_KEY + '&inc_color_details=0'
    response = requests.get(inventory_parts_url).json()
    inventory_parts = response['results']
    for part in inventory_parts:
        set_numbers.append(set)
        part_numbers.append(part['part']['part_num'])
        quantities.append(part['quantity'])


inventory_part_dictionary = {'set_number': set_numbers,
             'part_number': part_numbers, 'quantity': quantities}
import pandas as pd

df = pd.DataFrame(inventory_part_dictionary)
df


,set_number,part_number,quantity
0,30433-1,50335,1
1,30433-1,53923,1
2,30433-1,11478,2
3,30433-1,32348,2
4,30433-1,32526,2
...,...,...,...
1386,42145-1,32073,27
1387,42145-1,60485,5
1388,42145-1,32013,8
1389,42145-1,6536,27
